In [14]:
MODEL = "gemma2"
EMBEDDING_MODEL = "nomic-embed-text"

In [15]:
from  langchain_ollama.chat_models import ChatOllama
from langchain_ollama.embeddings import OllamaEmbeddings

In [16]:
model = ChatOllama(
  model=MODEL,
  temperature=0
) 

embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)

In [17]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
# chain.invoke("what is the solar system?")

In [18]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("2008_Mumbai_attacks.pdf")
pages = loader.load_and_split()
# pages

In [19]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [20]:
chain = prompt | model | parser

In [21]:
chain.invoke({
  "context": "I was given name dishant miyani and i was born in surat gujarat india",
  "question": "where was dishant born?"
})

'Surat, Gujarat, India \n'

In [22]:
from langchain.vectorstores import DocArrayInMemorySearch

vector_store = DocArrayInMemorySearch.from_documents(
  pages,
  embedding=embeddings
)

In [23]:
retriever = vector_store.as_retriever()

retriever.invoke("who is ajmal kasab?")

[Document(metadata={'source': '2008_Mumbai_attacks.pdf', 'page': 35}, page_content='159. Ramesh, Randeep; Duncan, Campbell; Paul, Lewis (28 November 2008). "They were in no\nhurry. Cool and composed, they killed and killed" (https://www.theguardian.com/world/2008/\nnov/28/mumbai-terror-attacks-india). The Guardian. London, UK. Archived (https://web.archi\nve.org/web/20081201071600/http://www.guardian.co.uk/world/2008/nov/28/mumbai-terror-a\nttacks-india) from the original on 1 December 2008. Retrieved 29 November 2008.\n160. Balakrishnan, Angela (28 November 2008). "Claims emerge of British terrorists in Mumbai"\n(https://www.theguardian.com/world/2008/nov/28/british-terrorists-mumbai). The Guardian.\nLondon, UK. Archived (https://web.archive.org/web/20081201071534/http://www.guardian.c\no.uk/world/2008/nov/28/british-terrorists-mumbai) from the original on 1 December 2008.\nRetrieved 29 November 2008.\n161. Morgan, Tom (28 November 2008). "Arrested Mumbai gunmen \'of British descent\'

In [24]:
from operator import itemgetter

chain = (
  {
    "context" : itemgetter("question") | retriever,
    "question" : itemgetter("question")
  }
  | prompt
  | model
  | parser
)

# for response in chain.stream({ "question": "what are the series of events in the attack" }):
#   print(response)
print(chain.invoke({ "question": "what are the series of events in the attack" }))

KeyboardInterrupt: 